<a href="https://colab.research.google.com/github/jyukki97/deeplearning/blob/main/bert_test_naver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-1fgqprm5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-1fgqprm5
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=f19d0909f5de6767f719229606d3c21d55e7a8e74c7560b6a1954059f96a25ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-5j73ubko/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [3]:
!pip install mxnet

In [4]:
!pip install transformers==3

In [5]:
!pip install sentencepiece

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [7]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [8]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
##GPU 사용 시
device = torch.device("cuda:0")

In [10]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [11]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-04-24 13:32:18--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-04-24 13:32:18--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7ccdbab4a425e7318ae16bb888.dl.dropboxusercontent.com/cd/0/get/BNNqAsg35SHTM5MdjcBitbePaNnlpGnRHn5H0J14oIg65gDSkNp05abS7hgjeiy5qkqKHAUpV4C6GWYQZ_d0UUOt4g0rvOkJ2_PTr2Glq4LCEWJBPTeoeFDRg55W2qEQW99YcMX5kNod5XZwyMWDWyox/file?dl=1# [following]
--2021-04-24 13:32:19--  https://uc7ccdbab4a425e7318ae16bb888.dl.dropboxusercontent.com/cd/0/get/BNNqAsg35SHTM5MdjcBitbePaNnlpGnRHn5H0J14oIg65gDSkNp05abS7hgjeiy5qkqKHA

In [12]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [13]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [15]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [20]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [21]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [22]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [23]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [24]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [25]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7222471833229065 train acc 0.53125
epoch 1 batch id 201 loss 0.5123291611671448 train acc 0.550839552238806
epoch 1 batch id 401 loss 0.49648383259773254 train acc 0.6717581047381546
epoch 1 batch id 601 loss 0.4470402002334595 train acc 0.7266794925124792
epoch 1 batch id 801 loss 0.4243738651275635 train acc 0.7575491573033708
epoch 1 batch id 1001 loss 0.3419242799282074 train acc 0.7750999000999002
epoch 1 batch id 1201 loss 0.3947815001010895 train acc 0.7887957951706911
epoch 1 batch id 1401 loss 0.3891148865222931 train acc 0.7982690935046396
epoch 1 batch id 1601 loss 0.28005409240722656 train acc 0.8064783728919426
epoch 1 batch id 1801 loss 0.2768963575363159 train acc 0.8132027345918934
epoch 1 batch id 2001 loss 0.28188514709472656 train acc 0.8190279860069966
epoch 1 batch id 2201 loss 0.30874103307724 train acc 0.8241211381190368

epoch 1 train acc 0.8279361312571104



epoch 1 test acc 0.8838914641943734


epoch 2 batch id 1 loss 0.45071831345558167 train acc 0.84375
epoch 2 batch id 201 loss 0.24185334146022797 train acc 0.8813743781094527
epoch 2 batch id 401 loss 0.24728497862815857 train acc 0.8832216334164589
epoch 2 batch id 601 loss 0.48860257863998413 train acc 0.8869592346089851
epoch 2 batch id 801 loss 0.331289678812027 train acc 0.8895521223470662
epoch 2 batch id 1001 loss 0.26052796840667725 train acc 0.8919517982017982
epoch 2 batch id 1201 loss 0.23229095339775085 train acc 0.8943068276436303
epoch 2 batch id 1401 loss 0.287630170583725 train acc 0.896368665239115
epoch 2 batch id 1601 loss 0.24931955337524414 train acc 0.8979348844472205
epoch 2 batch id 1801 loss 0.16027125716209412 train acc 0.899691143808995
epoch 2 batch id 2001 loss 0.20357732474803925 train acc 0.9017444402798601
epoch 2 batch id 2201 loss 0.2754301428794861 train acc 0.9033109950022717

epoch 2 train acc 0.9047168302047781



epoch 2 test acc 0.8883871483375959


...


epoch 3 test acc 0.8953204923273658


epoch 4 batch id 1 loss 0.30535629391670227 train acc 0.875
epoch 4 batch id 201 loss 0.10420223325490952 train acc 0.9557680348258707
epoch 4 batch id 401 loss 0.05538221448659897 train acc 0.9576449501246883
epoch 4 batch id 601 loss 0.25450217723846436 train acc 0.9585066555740432
epoch 4 batch id 801 loss 0.24888163805007935 train acc 0.9598938826466916
epoch 4 batch id 1001 loss 0.1532861292362213 train acc 0.9609765234765235
epoch 4 batch id 1201 loss 0.02544972486793995 train acc 0.9622580141548709
epoch 4 batch id 1401 loss 0.13214366137981415 train acc 0.9628278907922913
epoch 4 batch id 1601 loss 0.11925701051950455 train acc 0.9633725015615241
epoch 4 batch id 1801 loss 0.0809224545955658 train acc 0.9642039144919489
epoch 4 batch id 2001 loss 0.11729460954666138 train acc 0.9650565342328835
epoch 4 batch id 2201 loss 0.05838438868522644 train acc 0.9655270331667424

epoch 4 train acc 0.9660636198805461



epoch 4 test acc 0.8977581521739131


epoch 5 batch id 1 loss 0.37491610646247864 train acc 0.875
epoch 5 batch id 201 loss 0.054421477019786835 train acc 0.9738028606965174
epoch 5 batch id 401 loss 0.018566802144050598 train acc 0.9755299251870324
epoch 5 batch id 601 loss 0.19664208590984344 train acc 0.9756135607321131
epoch 5 batch id 801 loss 0.19145193696022034 train acc 0.9754603620474407
epoch 5 batch id 1001 loss 0.09001938998699188 train acc 0.9763673826173827
epoch 5 batch id 1201 loss 0.012618543580174446 train acc 0.9765820149875104
epoch 5 batch id 1401 loss 0.018666859716176987 train acc 0.9765346181299072
epoch 5 batch id 1601 loss 0.0975475087761879 train acc 0.9766747345409119
epoch 5 batch id 1801 loss 0.07542914897203445 train acc 0.9773042754025542
epoch 5 batch id 2001 loss 0.11776429414749146 train acc 0.9775893303348325
epoch 5 batch id 2201 loss 0.05972638353705406 train acc 0.9777941844616084

epoch 5 train acc 0.9780890038395904



epoch 5 test acc 0.8984375


In [26]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [94]:
test_dataloader2 = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [97]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader2)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  print(out, dataset_test[batch_id])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


tensor([[-2.6889,  2.8365]], device='cuda:0', grad_fn=<AddmmBackward>) ['굳 ㅋ', '1']
tensor([[-1.5314,  1.8185]], device='cuda:0', grad_fn=<AddmmBackward>) ['GDNTOPCLASSINTHECLUB', '0']
tensor([[ 2.0380, -1.6297]], device='cuda:0', grad_fn=<AddmmBackward>) ['뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', '0']
tensor([[ 3.4666, -3.3040]], device='cuda:0', grad_fn=<AddmmBackward>) ['지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '0']
tensor([[ 1.0047, -0.5317]], device='cuda:0', grad_fn=<AddmmBackward>) ['3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', '0']
tensor([[-2.8366,  2.9385]], device='cuda:0', grad_fn=<AddmmBackward>) ['음악이 주가 된, 최고의 음악영화', '1']
tensor([[ 3.4848, -3.3413]], device='cuda:0', grad_fn=<AddmmBackward>) ['진정한 쓰레기', '0']
tensor([[ 3.4778, -3.3179]], device='cuda:0', grad_fn=<AddmmBackward>) ['마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다', '0']
tensor([[ 3.4911, -3.3714]], device='cuda:0', grad_fn=<AddmmBackward>) ['갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류

KeyboardInterrupt: ignored

In [59]:
  max_vals, max_indices = torch.max(out, 1)
  a = (max_indices == label).sum().data.cpu().numpy()/max_indices.size()[0]
  a

RuntimeError: ignored

In [75]:
torch.save(model.state_dict(), "./naver_bert_model.pt")